In [1]:
import pandas as pd
import numpy as np
import json
import glob, os, subprocess
import matplotlib.pyplot as plt


In [2]:
summarized_df = pd.read_csv('/home/kin672/gentb-summer22/Creating Summaries from Relevant Files/7-18 Output/summarized_full_7-18.csv', usecols = [1, 2, 3, 4, 5])


In [3]:
x = summarized_df.loc[:, 'Lineage'].str.split(',', expand=True)
summarized_df['Number of Lineages'] = pd.Series()
for index, row in x.iterrows():
    summarized_df.at[index, 'Number of Lineages'] = len(row.unique()) - 1
    

/tmp/ipykernel_32467/281807930.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  summarized_df['Number of Lineages'] = pd.Series()


In [4]:
summarized_df.loc[summarized_df.loc[:,'Number of Lineages'] > 1, :]

,Drug,Lineage,Percentage of Strains that are Resistant,Number of Resistant Strains,Number of Susceptible Strains,Number of Lineages
3,amk,"1.1.1,4.9",0.0,0,1,2.0
6,amk,"1.1.1.1.1,2.2.1.1.1",0.0,0,3,2.0
8,amk,"1.1.1.2,4.2.1.1.1.1.1.1.i2",0.0,0,1,2.0
9,amk,"1.1.1.2.i1,2.2.1.1.1.i3",0.0,0,1,2.0
11,amk,"1.1.2,2.2.1.1.1",0.0,0,2,2.0
...,...,...,...,...,...,...
2197,str,"4.3.3,4.8",0.0,0,2,2.0
2207,str,"4.5,4.9",0.0,0,2,2.0
2215,str,"4.8,4.3.4.1",0.0,0,1,2.0
2216,str,"4.8,4.9",0.0,0,1,2.0


In [5]:
y = summarized_df.loc[summarized_df.Drug == 'amk', :].iloc[:, [1, 3, 4, 5]].groupby(['Number of Lineages'], groups = '.drop').sum().reset_index()
y = y.assign(Total = [y['Number of Resistant Strains'][i] + y['Number of Susceptible Strains'][i] for i in range(len(y))])
y

TypeError: DataFrame.groupby() got an unexpected keyword argument 'groups'

In [ ]:
y.to_csv('/home/kin672//Kira_GenTB/Distribution of Lineages/Num_Lineages.csv', index = False)
y

In [7]:
single_lineage = summarized_df.loc[summarized_df['Number of Lineages'] == 1, :].reset_index()
single_lineage = single_lineage.assign(Main_Lineage = [long[0:1] for long in single_lineage.Lineage])
single_lineage = single_lineage.assign(Total_Isolates = [single_lineage['Number of Resistant Strains'][i] + single_lineage['Number of Susceptible Strains'][i] for i in range(len(single_lineage))])
single_lineage

,index,Drug,Lineage,Percentage of Strains that are Resistant,Number of Resistant Strains,Number of Susceptible Strains,Number of Lineages,Main_Lineage,Total_Isolates
0,0,amk,1,0.000000,0,1,1.0,1,1
1,1,amk,1.1,0.000000,0,1,1.0,1,1
2,2,amk,1.1.1,0.000000,0,3,1.0,1,3
3,4,amk,1.1.1.1,0.000000,0,3,1.0,1,3
4,5,amk,1.1.1.1.1,0.000000,0,23,1.0,1,23
...,...,...,...,...,...,...,...,...,...
1334,2214,str,4.8,19.402985,78,324,1.0,4,402
1335,2217,str,4.9,14.062500,9,55,1.0,4,64
1336,2218,str,5,0.000000,0,1,1.0,5,1
1337,2219,str,6,0.000000,0,1,1.0,6,1


In [8]:
single_lineage.loc[single_lineage.Drug == 'amk', ['Main_Lineage', 'Total_Isolates']].groupby(['Main_Lineage']).sum()

,Total_Isolates
Main_Lineage,
1,391
2,3551
3,355
4,3672
5,1
6,1
B,20


In [9]:
table = single_lineage.loc[single_lineage.Drug == 'amk', ['Lineage', 'Total_Isolates']].groupby(['Lineage']).sum()
# table.to_csv('/home/kin672//Kira_GenTB/Distribution of Lineages/Num_per_Lineage.csv')

In [10]:
table

,Total_Isolates
Lineage,
1,1
1.1,1
1.1.1,3
1.1.1.1,3
1.1.1.1.1,23
...,...
4.8,402
4.9,64
5,1


In [11]:
## BEGIN COMPUTING RESISTANCE PROFILE
full_df = pd.read_csv('/home/kin672/gentb-summer22/Creating Summaries from Relevant Files/7-18 Output/full_df_7-18.csv')

In [12]:
def main_lin(x):
    x = str(x)
    if ',' in x:
        return 'Multiple'
    elif x == 'BOV_AFRI':
        return 'Bovine'
    elif pd.isna(x):
        return 'N/A'
    else:
        return x[0:1]

summarized_df = summarized_df.assign(Main_Lineage = [main_lin(long) for long in summarized_df.Lineage])


In [13]:
summarized_df

,Drug,Lineage,Percentage of Strains that are Resistant,Number of Resistant Strains,Number of Susceptible Strains,Number of Lineages,Main_Lineage
0,amk,1,0.0,0,1,1.0,1
1,amk,1.1,0.0,0,1,1.0,1
2,amk,1.1.1,0.0,0,3,1.0,1
3,amk,"1.1.1,4.9",0.0,0,1,2.0,Multiple
4,amk,1.1.1.1,0.0,0,3,1.0,1
...,...,...,...,...,...,...,...
2218,str,5,0.0,0,1,1.0,5
2219,str,6,0.0,0,1,1.0,6
2220,str,"6,BOV_AFRI",0.0,0,4,2.0,Multiple
2221,str,BOV_AFRI,0.0,0,20,1.0,Bovine


In [ ]:
single_lineage = single_lineage.assign(Main_Lineag = [long[0:1] for long in single_lineage.Lineage])
